# Problem Statement:

* Create Dynamic Pricing model for different locations and vehicle types, hourly and daily prices.

# Know the Data

* `booking_billed_duration`:
    1. This is in minutes.
    2. For better understanding consider converting it to hour. The formula is:
        `booking_billed_duration (in hours) = booking_billed_duration / 60`
    3. However, this is already done and result is stored in `booking_rates_hours`, `booking_rates_24hours`, `booking_rates_overnight`.
    4. If hours are less than 8 then it is stored in `booking_rates_hours` else `booking_rates_24hours`.
* `booking_mileage`:
    1. This is in miles.

* `booking_actual_cost_distance`:
    1. This is in Pound and Pence
    2. The formula is:
        ``booking_mileage * per mile (Petrol / EV)``

* `booking_actual_cost_time`:
    1. This is in Pound and Pence.
    2. The formula is:
        ``booking_rates_hours (or) booking_rates_24hours` * hourly_rates (or) daily_rates`

        _NOTE: Add columns hourly_rates and daily_rates_

# 1. Data Cleaning

## 1.1. Load libraries

In [1]:
# Import required libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# Display all the columns of the dataframe
pd.pandas.set_option('display.max_columns', None)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### 1.2. Load data

In [3]:
# Data location
DATA_PATH = "/content/drive/MyDrive/MSc Dissertation/data/"

In [4]:
# Load all files

bookings_2019_df = pd.read_csv(f"{DATA_PATH}2019 Bookings.csv", low_memory=False)
bookings_2020_df = pd.read_csv(f"{DATA_PATH}2020 Bookings.csv", low_memory=False)
bookings_2021_df = pd.read_csv(f"{DATA_PATH}2021 Bookings.csv", low_memory=False)
bookings_2022_df = pd.read_csv(f"{DATA_PATH}2022 Bookings.csv", low_memory=False)
bookings_2023_df = pd.read_csv(f"{DATA_PATH}2023 Bookings.csv", low_memory=False)
bookings_2024_df = pd.read_csv(f"{DATA_PATH}2024 Bookings.csv", low_memory=False)
tariff_df = pd.read_csv(f"{DATA_PATH}Diff Tariffs.csv", low_memory=False)


In [5]:
# Merge all files into one dataframe
bookings_df = pd.concat([bookings_2019_df, bookings_2020_df, bookings_2021_df, bookings_2022_df, bookings_2023_df, bookings_2024_df])

# Print the shape of the dataframe
print(f"The shape of the merged dataframe: {bookings_df.shape}")

In [7]:
# Reset index
bookings_df.reset_index(drop=True, inplace=True)

In [8]:
# Look at the data
bookings_df.head()

,account_id,Contract,user_id,vehicle_description,vehicle_registration,vehicle_communication_id,vehicle_operator_name,vehicle_office_use,location_description,location_office_use,booking_id,booking_reservation,booking_maintenance,booking_start,booking_end,booking_duration,booking_actual_start,booking_actual_end,booking_actual_duration,booking_billed_start,booking_billed_end,booking_billed_duration,booking_mileage,booking_tariff,booking_rates_hours,booking_rates_24hours,booking_rates_overnight,booking_estimated_cost,booking_actual_cost_distance,booking_actual_cost_time,booking_actual_cost_total,booking_credits_used,booking_transactions_value,booking_total_paid,booking_status,booking_ended_early,booking_created_at,booking_cancelled_at,booking_cancellation_reason
0,9ef6f9f2-bb87-11e8-8478-3392d6ed7026,PAYG,5cab890a-bb87-11e8-8c45-8b43f7434530,RETIRED- White Toyota Yaris Hybrid NJ68 VOP Au...,NJ68 VOP,NaN,Co-wheels,PAYG,NaN,NaN,ac2d5146-2cee-11ea-a00a-57b3c15418c7,26692666,no,01/01/2020 23:45,01/02/2020 0:15,30,01/02/2020 0:11,01/02/2020 0:35,24.0,01/01/2020 23:45,01/02/2020 0:45,60,16.0,Co-wheels Standard Medium,1.0,0,0,2.75,2.88,5.5,8.38,0,-8.38,8.38,ended,no,01/01/2020 23:30,NaN,NaN
1,837460ea-1a02-11ea-9ddd-919b17a2c51c,PAYG,e4f9362e-1822-11ea-8de1-473e949dd300,RETIRED - White Toyota Aygo X-Play Nav (NL66 NKZ),NL66 NKZ,NaN,Co-wheels,PAYG,Lower Maudlin DNU,NaN,fc59ba9c-2ce9-11ea-8efc-bfa292f04054,26692660,no,01/01/2020 23:00,01/02/2020 8:00,540,01/01/2020 23:06,01/02/2020 1:24,138.0,01/01/2020 23:00,01/02/2020 8:00,540,82.0,Co-wheels Standard Small,0.0,0,1,9.50,14.76,9.5,24.26,0,-24.26,24.26,ended,no,01/01/2020 22:56,NaN,NaN
2,bcdf83ac-cf78-11e9-a7b5-eba9440aff95,PAYG,5b58ca44-cf78-11e9-8a88-f32ff93f96dc,RETIRED - Renault Zoe EV (DL68 LNK),DL68 LNK,NaN,Co-wheels,PAYG,Bristol - Effingham Road,BRI018,1cf7fae6-2ce7-11ea-a970-fbf7b3786523,26692658,no,01/01/2020 22:45,01/02/2020 2:30,225,01/01/2020 22:46,01/02/2020 2:01,195.0,01/01/2020 22:45,01/02/2020 2:30,225,5.0,Co-wheels Standard Electric,0.0,0,1,11.00,0.0,11.0,11.0,0,-11.00,11.00,ended,no,01/01/2020 22:36,NaN,NaN
3,10d4ff50-c68a-490b-9831-73a2a53a789e,Franchise,27a79ce4-eeff-4a45-9727-cff211058712,RETIRED - Toyota Yaris Hybrid NAV (NJ18 VTX),NJ18 VTX,NaN,Co-wheels Birmingham,PAYG,NaN,NaN,887d643e-2ce9-11ea-be0d-d996759c0cec,26692659,no,01/01/2020 22:45,01/02/2020 22:45,1440,01/01/2020 22:54,01/02/2020 22:30,1416.0,01/01/2020 22:45,01/02/2020 22:45,1440,112.0,Co-wheels Standard Medium,0.0,1,0,38.50,20.16,38.5,58.66,0,-83.66,0.00,ended,yes,01/01/2020 22:53,NaN,NaN
4,ad4720a8-371c-11e8-9c54-45ef183a979b,Contract,84845dc8-a7cb-11e8-a9fa-4fcddbe2d33e,RETIRED - SEAT Ateca 4X4 (NG18 EKY),NG18 EKY,NaN,Co-wheels,Pool Car - Telematics,North Shields - North Tyneside General Hospital,NTH006,33c8d096-2b58-11ea-9f3f-5f76ab89e87a,25691980,no,01/01/2020 22:30,01/02/2020 8:45,615,01/01/2020 22:49,01/02/2020 8:29,580.0,01/01/2020 22:30,01/02/2020 8:45,615,40.0,Billing Exempt (24/7) (Default),10.25,0,0,0.00,0.0,0.0,0.0,0,0.00,0.00,ended,no,12/30/2019 23:00,NaN,NaN


In [9]:
# Select only PAYG fleets
bookings_filtered_df = bookings_df[bookings_df['Contract'] == 'PAYG']
bookings_filtered_df.head()

,account_id,Contract,user_id,vehicle_description,vehicle_registration,vehicle_communication_id,vehicle_operator_name,vehicle_office_use,location_description,location_office_use,booking_id,booking_reservation,booking_maintenance,booking_start,booking_end,booking_duration,booking_actual_start,booking_actual_end,booking_actual_duration,booking_billed_start,booking_billed_end,booking_billed_duration,booking_mileage,booking_tariff,booking_rates_hours,booking_rates_24hours,booking_rates_overnight,booking_estimated_cost,booking_actual_cost_distance,booking_actual_cost_time,booking_actual_cost_total,booking_credits_used,booking_transactions_value,booking_total_paid,booking_status,booking_ended_early,booking_created_at,booking_cancelled_at,booking_cancellation_reason
0,9ef6f9f2-bb87-11e8-8478-3392d6ed7026,PAYG,5cab890a-bb87-11e8-8c45-8b43f7434530,RETIRED- White Toyota Yaris Hybrid NJ68 VOP Au...,NJ68 VOP,NaN,Co-wheels,PAYG,NaN,NaN,ac2d5146-2cee-11ea-a00a-57b3c15418c7,26692666,no,01/01/2020 23:45,01/02/2020 0:15,30,01/02/2020 0:11,01/02/2020 0:35,24.0,01/01/2020 23:45,01/02/2020 0:45,60,16.0,Co-wheels Standard Medium,1.0,0,0,2.75,2.88,5.5,8.38,0,-8.38,8.38,ended,no,01/01/2020 23:30,NaN,NaN
1,837460ea-1a02-11ea-9ddd-919b17a2c51c,PAYG,e4f9362e-1822-11ea-8de1-473e949dd300,RETIRED - White Toyota Aygo X-Play Nav (NL66 NKZ),NL66 NKZ,NaN,Co-wheels,PAYG,Lower Maudlin DNU,NaN,fc59ba9c-2ce9-11ea-8efc-bfa292f04054,26692660,no,01/01/2020 23:00,01/02/2020 8:00,540,01/01/2020 23:06,01/02/2020 1:24,138.0,01/01/2020 23:00,01/02/2020 8:00,540,82.0,Co-wheels Standard Small,0.0,0,1,9.50,14.76,9.5,24.26,0,-24.26,24.26,ended,no,01/01/2020 22:56,NaN,NaN
2,bcdf83ac-cf78-11e9-a7b5-eba9440aff95,PAYG,5b58ca44-cf78-11e9-8a88-f32ff93f96dc,RETIRED - Renault Zoe EV (DL68 LNK),DL68 LNK,NaN,Co-wheels,PAYG,Bristol - Effingham Road,BRI018,1cf7fae6-2ce7-11ea-a970-fbf7b3786523,26692658,no,01/01/2020 22:45,01/02/2020 2:30,225,01/01/2020 22:46,01/02/2020 2:01,195.0,01/01/2020 22:45,01/02/2020 2:30,225,5.0,Co-wheels Standard Electric,0.0,0,1,11.00,0.0,11.0,11.0,0,-11.00,11.00,ended,no,01/01/2020 22:36,NaN,NaN
6,29f5ecce-4a9d-11e6-a70a-f39821a3496c,PAYG,622754ee-4a9c-11e6-a82f-4ba9e3c70859,RETIRED - BMW i3 Electric Vehicle (Automatic),NL69 JZM,1343.0,Co-wheels,PAYG - Funded,NaN,NaN,513090a0-2ce4-11ea-9bfc-a300cc72d38e,25692652,no,01/01/2020 22:15,01/02/2020 8:00,585,01/01/2020 22:22,01/02/2020 6:44,502.0,01/01/2020 22:15,01/02/2020 8:00,585,113.0,Co-wheels Standard Electric,0.0,0,1,11.00,0.0,11.0,11.0,0,-11.00,11.00,ended,no,01/01/2020 22:16,NaN,NaN
7,68c151d8-0d0d-11ea-bed8-ebcb87cb5715,PAYG,efa7004a-0d0c-11ea-b0b4-590f743c744f,White Toyota Yaris Petrol Hybrid (Automatic) -...,ND19 HWY,1211.0,Co-wheels,PAYG - Funded,Newbury - Boundary Road,NEW001,591b7a98-2cd8-11ea-a341-71665a42c0e8,25692603,no,01/01/2020 21:30,01/01/2020 23:30,120,01/01/2020 21:29,01/01/2020 23:28,119.0,01/01/2020 21:30,01/01/2020 23:30,120,6.0,Co-wheels Standard Medium,2.0,0,0,11.00,1.08,11.0,12.08,0,-12.08,11.00,ended,no,01/01/2020 20:50,NaN,NaN


In [10]:
# Print the shape of the dataframe for PAYG fleets
print(f"The shape of the PAYG fleet dataframe: {bookings_filtered_df.shape}")

The shape of the PAYG fleet dataframe: (436354, 39)


### 1.3. Handling Missing Values

In [11]:
bookings_filtered_df.isnull().sum()

account_id                           0
Contract                             0
user_id                              0
vehicle_description                 28
vehicle_registration                28
vehicle_communication_id        139683
vehicle_operator_name                0
vehicle_office_use                  28
location_description             55311
location_office_use              56958
booking_id                           0
booking_reservation                  0
booking_maintenance                  0
booking_start                        0
booking_end                          0
booking_duration                     0
booking_actual_start                 5
booking_actual_end                   0
booking_actual_duration              5
booking_billed_start                 0
booking_billed_end                   0
booking_billed_duration              0
booking_mileage                     50
booking_tariff                      29
booking_rates_hours                  0
booking_rates_24hours    

#### 1.3.1. Vehicle Description

In [12]:
# Check columns vehicle_description for missing values
bookings_filtered_df[bookings_filtered_df['vehicle_description'].isnull()]

,account_id,Contract,user_id,vehicle_description,vehicle_registration,vehicle_communication_id,vehicle_operator_name,vehicle_office_use,location_description,location_office_use,booking_id,booking_reservation,booking_maintenance,booking_start,booking_end,booking_duration,booking_actual_start,booking_actual_end,booking_actual_duration,booking_billed_start,booking_billed_end,booking_billed_duration,booking_mileage,booking_tariff,booking_rates_hours,booking_rates_24hours,booking_rates_overnight,booking_estimated_cost,booking_actual_cost_distance,booking_actual_cost_time,booking_actual_cost_total,booking_credits_used,booking_transactions_value,booking_total_paid,booking_status,booking_ended_early,booking_created_at,booking_cancelled_at,booking_cancellation_reason
359197,eebb601e-bed7-11e9-8f72-fbf50f6fba82,PAYG,9946df28-bed7-11e9-be05-1de26f660be2,NaN,NaN,NaN,unknown,NaN,NaN,NaN,6255aa58-8f23-11eb-891b-d75469a58a71,80867460,no,3/30/2021 9:15,3/30/2021 11:45,150,3/30/2021 9:30,3/31/2021 9:30,1440.0,3/30/2021 9:15,3/31/2021 9:30,1455,0.0,NaN,0,0,0,13.75,0,0,0,0,0.00,0.00,ended,no,3/27/2021 17:39,NaN,NaN
359845,8dd6169c-57d6-11e6-9e81-09a7d1307426,PAYG,1ba4c8d4-57d6-11e6-aa64-39cb3df7f74f,NaN,NaN,NaN,unknown,NaN,NaN,NaN,23389d4a-8f3b-11eb-91b6-25a02d337597,80867531,no,3/28/2021 9:00,3/29/2021 9:00,1440,3/28/2021 9:20,3/28/2021 12:28,188.0,3/28/2021 9:00,3/29/2021 9:00,1440,26.0,NaN,0,0,0,38.50,0,0,0,0,0.00,0.00,ended,no,3/27/2021 20:29,NaN,NaN
359882,1dd623d0-f0a4-11ea-9ad6-2794ee5db418,PAYG,c2f1e102-f0a3-11ea-a40a-9d81215445cb,NaN,NaN,NaN,unknown,NaN,NaN,NaN,65af4494-8f40-11eb-8cc8-f33fd4397ef0,80867544,no,3/27/2021 21:45,3/27/2021 22:45,60,3/27/2021 21:41,3/27/2021 22:59,78.0,3/27/2021 21:45,3/27/2021 23:00,75,12.0,NaN,0,0,0,7.25,0,0,0,0,-7.25,7.25,ended,no,3/27/2021 21:07,NaN,NaN
359883,60f3c296-16ae-11eb-805b-53e66836bd2c,PAYG,ec3d526e-16ad-11eb-946a-3f0727890e08,NaN,NaN,NaN,unknown,NaN,NaN,NaN,5c418ce6-8f40-11eb-8363-7537e29d5f85,80867543,no,3/27/2021 21:15,3/27/2021 22:45,90,3/27/2021 21:12,3/27/2021 22:38,86.0,3/27/2021 21:15,3/27/2021 22:45,90,46.0,NaN,0,0,0,9.38,0,0,0,0,-9.38,9.38,ended,no,3/27/2021 21:06,NaN,NaN
359884,58093fab-a84b-4aa7-9664-5dbd6e367907,PAYG,590bc43d-184f-40a0-9429-d10d83b4c953,NaN,NaN,NaN,unknown,NaN,NaN,NaN,8157503a-8f3e-11eb-8011-cda35da4470e,80867540,no,3/27/2021 21:15,3/27/2021 21:30,15,3/27/2021 21:15,3/27/2021 21:28,13.0,3/27/2021 21:15,3/27/2021 21:30,15,3.0,NaN,0,0,0,1.49,0,0,0,0,-2.12,0.00,ended,no,3/27/2021 20:53,NaN,NaN
359885,57ef93ea-80ea-11eb-9907-43a25a7d47ae,PAYG,b6f14c40-80e9-11eb-97e7-f9a2868f06ed,NaN,NaN,NaN,unknown,NaN,NaN,NaN,a1f7ab2e-8f3d-11eb-8811-1b60b3b7cb68,80867539,no,3/27/2021 21:15,3/28/2021 6:00,465,3/27/2021 21:25,3/28/2021 3:51,326.0,3/27/2021 21:15,3/28/2021 6:00,465,14.0,NaN,0,0,0,16.50,0,0,0,0,-16.50,16.50,ended,yes,3/27/2021 20:47,NaN,NaN
359888,53bd4edc-caf0-11e8-87ed-5be032df60ca,PAYG,dfa906c6-caef-11e8-883d-0b801a6b931d,NaN,NaN,NaN,unknown,NaN,NaN,NaN,73bb826e-8f3b-11eb-9b86-572a63a6cc2e,80867534,no,3/27/2021 20:30,3/27/2021 21:30,60,3/27/2021 20:36,3/27/2021 21:23,47.0,3/27/2021 20:30,3/27/2021 21:30,60,5.0,NaN,0,0,0,3.60,0,0,0,0,-2.40,2.40,ended,no,3/27/2021 20:31,NaN,NaN
359889,11cee930-0014-11eb-a5c9-538de04c25e4,PAYG,da0bf182-0013-11eb-a6ce-cd86864c31c8,NaN,NaN,NaN,unknown,NaN,NaN,NaN,d93555ce-8f37-11eb-be2d-6d15638694be,80867514,no,3/27/2021 20:15,3/28/2021 10:00,765,3/27/2021 20:25,3/28/2021 10:22,777.0,3/27/2021 20:15,3/28/2021 10:30,795,38.0,NaN,0,0,0,33.00,0,0,0,0,-33.00,33.00,ended,no,3/27/2021 20:05,NaN,NaN
359891,7765d12a-179a-11e7-b53a-bd3b48518386,PAYG,a9f21348-1799-11e7-8471-1188e7e7dfaa,NaN,NaN,NaN,unknown,NaN,NaN,NaN,4f10bbea-8f32-11eb-9de9-c5918cc6da54,80867507,no,3/27/2021 20:00,3/28/2021 0:00,240,3/27/2021 20:09,3/27/2021 22:37,148.0,3/27/2021 20:00,3/28/2021 0:00,240,34.0,NaN,0,0,0,11.00,0,0,0,0,-11.00,11.00,ended,no,3/27/2021 19:26,NaN,NaN
359892,d6d5cafe-54f8-11ea-9e37-279c5ed60562,PAYG,91b04bb6-54f8-11ea-a34b-2767694e0207,NaN,NaN,NaN,unknown,

From above it can be seen that the column names 'vehicle_description', 'vehicle_registration', 'vehicle_operator_name', 'vehicle_office_use', 'booking_tariff', 'booking_rates_hours', 'booking_rates_24hours', 'booking_rates_overnight', these important informations are missing hence consider dropping these rows.

In [13]:
# Drop rows where 'location_description' is missing
bookings_filtered_df = bookings_filtered_df.dropna(subset=['vehicle_description'])

In [14]:
bookings_filtered_df.isnull().sum()

account_id                           0
Contract                             0
user_id                              0
vehicle_description                  0
vehicle_registration                 0
vehicle_communication_id        139655
vehicle_operator_name                0
vehicle_office_use                   0
location_description             55285
location_office_use              56932
booking_id                           0
booking_reservation                  0
booking_maintenance                  0
booking_start                        0
booking_end                          0
booking_duration                     0
booking_actual_start                 5
booking_actual_end                   0
booking_actual_duration              5
booking_billed_start                 0
booking_billed_end                   0
booking_billed_duration              0
booking_mileage                     50
booking_tariff                       1
booking_rates_hours                  0
booking_rates_24hours    

#### 1.3.2. Booking Actual Start and Booking Actual Duration

In [15]:
bookings_filtered_df[bookings_filtered_df['booking_actual_start'].isnull()]

,account_id,Contract,user_id,vehicle_description,vehicle_registration,vehicle_communication_id,vehicle_operator_name,vehicle_office_use,location_description,location_office_use,booking_id,booking_reservation,booking_maintenance,booking_start,booking_end,booking_duration,booking_actual_start,booking_actual_end,booking_actual_duration,booking_billed_start,booking_billed_end,booking_billed_duration,booking_mileage,booking_tariff,booking_rates_hours,booking_rates_24hours,booking_rates_overnight,booking_estimated_cost,booking_actual_cost_distance,booking_actual_cost_time,booking_actual_cost_total,booking_credits_used,booking_transactions_value,booking_total_paid,booking_status,booking_ended_early,booking_created_at,booking_cancelled_at,booking_cancellation_reason
251918,14ba9e1c-c90b-11eb-96fe-87afa0988722,PAYG,61579b68-c90a-11eb-9e19-3fed6f0afd7f,RETIRED - Mini One (Manual),NJ20 ZUD,1409.0,Co-wheels,PAYG,Newcastle - Eslington Terrace (Jesmond),NCL011,b7c7cdde-50f9-11ec-bd99-b3f7fca9bd7c,93166206,no,12/26/2021 9:00,1/3/2022 9:00,11520,NaN,1/3/2022 9:00,NaN,12/26/2021 9:00,1/3/2022 9:00,11520,NaN,Co-wheels Standard Medium,0,8,0,308.00,0,308,308,0,0.00,0.00,cancelled,no,11/29/2021 9:50,12/23/2021 10:44,swapped
454501,ab09e586-7030-11e7-8c9e-e152f089898e,PAYG,89c66fd0-702f-11e7-a848-69f0b6c05137,RETIRED - Mini One (Manual),NK20 UHR,1399.0,Co-wheels,PAYG,Dundee - Hilltown Zone,DDE005,79828002-f1e2-11ec-a15d-c90c62cd9d36,93297854,no,6/22/2022 21:30,6/22/2022 23:30,120,NaN,6/22/2022 23:30,NaN,6/22/2022 21:30,6/22/2022 23:30,120,NaN,Co-wheels Standard Medium,2.0,0,0,11.50,0.0,11.5,11.5,0,-23.00,0.00,cancelled,no,6/22/2022 5:19,6/22/2022 15:37,NaN
487933,fdf87c28-6684-11eb-a9ff-1d29633aee3c,PAYG,510ea6d6-6684-11eb-bf01-47525b5a1040,RETIRED - BMW i3 Electric Vehicle (Automatic),NA20 DDO,1479.0,Co-wheels,PAYG,Dundee - Greenmarket Car Park,DDE017,998b94fc-9363-11ec-8c72-5be0ffc77cdf,93220200,no,3/28/2022 15:00,3/31/2022 15:00,4320,NaN,3/31/2022 15:00,NaN,3/28/2022 15:00,3/31/2022 15:00,4320,NaN,Scottish HA Electric,0.0,3,0,105.00,0.0,105.0,105.0,0,-105.00,0.00,cancelled,no,2/21/2022 22:14,3/28/2022 16:21,NaN
647783,1bbb688c-dd96-11eb-ab0d-8198d9c1643d,PAYG,5ec158e2-dd93-11eb-a17b-efad88107da9,White MG 3 Petrol (Manual) - ND71 KLJ,ND71 KLJ,2193.0,Co-wheels,PAYG,Newcastle - Dean Street,NCL008,89d52a4e-b222-11ed-b9d5-4b35caa9b46f,93449846,no,2/21/2023 20:00,2/21/2023 23:00,180,NaN,2/21/2023 23:00,NaN,2/21/2023 20:00,2/21/2023 23:00,180,NaN,Co-wheels Standard Medium,3.0,0,0,17.85,0.0,17.85,17.85,0,-17.85,17.85,expired,no,2/21/2023 20:01,NaN,NaN
654466,a27ee38e-ac2c-11ec-8434-0572151415be,PAYG,8ce2015e-ab65-11ec-bf37-236d15084145,White Toyota Yaris Petrol Hybrid (Automatic) -...,ND69 RMZ,1338.0,Co-wheels,PAYG,Hastings - Russell Street Car Park,HAS001,02f0688e-9fb9-11ed-bf0b-b5ca48132607,93434405,no,02/04/2023 10:00,02/04/2023 15:00,300,NaN,02/04/2023 15:00,NaN,02/04/2023 10:00,02/04/2023 15:00,300,NaN,Co-wheels Standard Medium,5.0,0,0,29.75,0.0,29.75,29.75,0,0.00,0.00,cancelled,no,1/29/2023 9:40,02/03/2023 19:36,NaN


Let's try to fill missing values for 'booking_actual_start' and 'booking_actual_duration', It'll make sense if we fill this missing values with 'booking_billed_start' and 'booking_billed_duration' columns repectively because these columns shows the actual data of the trip.

In [16]:
bookings_filtered_df['booking_actual_start'] = bookings_filtered_df['booking_actual_start'].fillna(bookings_filtered_df['booking_billed_start'])
bookings_filtered_df['booking_actual_duration'] = bookings_filtered_df['booking_actual_duration'].fillna(bookings_filtered_df['booking_billed_duration'])

In [17]:
bookings_filtered_df.isnull().sum()

account_id                           0
Contract                             0
user_id                              0
vehicle_description                  0
vehicle_registration                 0
vehicle_communication_id        139655
vehicle_operator_name                0
vehicle_office_use                   0
location_description             55285
location_office_use              56932
booking_id                           0
booking_reservation                  0
booking_maintenance                  0
booking_start                        0
booking_end                          0
booking_duration                     0
booking_actual_start                 0
booking_actual_end                   0
booking_actual_duration              0
booking_billed_start                 0
booking_billed_end                   0
booking_billed_duration              0
booking_mileage                     50
booking_tariff                       1
booking_rates_hours                  0
booking_rates_24hours    

#### 1.3.3. Booking Tariff

Let's drop row where 'booking_tariff' contains null value. I'm dropping this because there is only one row with null value in 'booking_tariff'. So even if I drop, it won't impact on the model.

In [18]:
bookings_filtered_df = bookings_filtered_df.dropna(subset=['booking_tariff'])

In [19]:
bookings_filtered_df.isnull().sum()

account_id                           0
Contract                             0
user_id                              0
vehicle_description                  0
vehicle_registration                 0
vehicle_communication_id        139655
vehicle_operator_name                0
vehicle_office_use                   0
location_description             55285
location_office_use              56932
booking_id                           0
booking_reservation                  0
booking_maintenance                  0
booking_start                        0
booking_end                          0
booking_duration                     0
booking_actual_start                 0
booking_actual_end                   0
booking_actual_duration              0
booking_billed_start                 0
booking_billed_end                   0
booking_billed_duration              0
booking_mileage                     50
booking_tariff                       0
booking_rates_hours                  0
booking_rates_24hours    

#### 1.3.4. Location Description and Location Office Use

Dropping the rows where both 'location_description' and 'location_office_use' contains NaN (missing) values. I'm not dropping rows where either of the mentioned columns are NaN value, because new column name 'location' can be introducted with either of the columns.

In [20]:
bookings_filtered_df = bookings_filtered_df.dropna(subset=['location_office_use', 'location_description'], how='all')

In [21]:
bookings_filtered_df.isnull().sum()

account_id                           0
Contract                             0
user_id                              0
vehicle_description                  0
vehicle_registration                 0
vehicle_communication_id         93961
vehicle_operator_name                0
vehicle_office_use                   0
location_description                 0
location_office_use               1647
booking_id                           0
booking_reservation                  0
booking_maintenance                  0
booking_start                        0
booking_end                          0
booking_duration                     0
booking_actual_start                 0
booking_actual_end                   0
booking_actual_duration              0
booking_billed_start                 0
booking_billed_end                   0
booking_billed_duration              0
booking_mileage                     47
booking_tariff                       0
booking_rates_hours                  0
booking_rates_24hours    

In the below code, after dropping rows where both 'location_description' and 'location_office_use' contains NaN (missing) values, I've only selected first three characters of the values of 'location_office_use', this is because this column contains various bays at one region (for example: Newcastle: NCL001, NCL003, etc..) and to solve the problem statement I will need only location (region) name.

In [22]:
# Select first 3 characters from the 'location_office_use',
# because later in the code I want to create location column based on this row so consider only vital information.

bookings_filtered_df['location_office_use'] = bookings_filtered_df['location_office_use'].str[:3]

In [23]:
bookings_filtered_df.head()

,account_id,Contract,user_id,vehicle_description,vehicle_registration,vehicle_communication_id,vehicle_operator_name,vehicle_office_use,location_description,location_office_use,booking_id,booking_reservation,booking_maintenance,booking_start,booking_end,booking_duration,booking_actual_start,booking_actual_end,booking_actual_duration,booking_billed_start,booking_billed_end,booking_billed_duration,booking_mileage,booking_tariff,booking_rates_hours,booking_rates_24hours,booking_rates_overnight,booking_estimated_cost,booking_actual_cost_distance,booking_actual_cost_time,booking_actual_cost_total,booking_credits_used,booking_transactions_value,booking_total_paid,booking_status,booking_ended_early,booking_created_at,booking_cancelled_at,booking_cancellation_reason
1,837460ea-1a02-11ea-9ddd-919b17a2c51c,PAYG,e4f9362e-1822-11ea-8de1-473e949dd300,RETIRED - White Toyota Aygo X-Play Nav (NL66 NKZ),NL66 NKZ,NaN,Co-wheels,PAYG,Lower Maudlin DNU,NaN,fc59ba9c-2ce9-11ea-8efc-bfa292f04054,26692660,no,01/01/2020 23:00,01/02/2020 8:00,540,01/01/2020 23:06,01/02/2020 1:24,138.0,01/01/2020 23:00,01/02/2020 8:00,540,82.0,Co-wheels Standard Small,0.0,0,1,9.50,14.76,9.5,24.26,0,-24.26,24.26,ended,no,01/01/2020 22:56,NaN,NaN
2,bcdf83ac-cf78-11e9-a7b5-eba9440aff95,PAYG,5b58ca44-cf78-11e9-8a88-f32ff93f96dc,RETIRED - Renault Zoe EV (DL68 LNK),DL68 LNK,NaN,Co-wheels,PAYG,Bristol - Effingham Road,BRI,1cf7fae6-2ce7-11ea-a970-fbf7b3786523,26692658,no,01/01/2020 22:45,01/02/2020 2:30,225,01/01/2020 22:46,01/02/2020 2:01,195.0,01/01/2020 22:45,01/02/2020 2:30,225,5.0,Co-wheels Standard Electric,0.0,0,1,11.00,0.0,11.0,11.0,0,-11.00,11.00,ended,no,01/01/2020 22:36,NaN,NaN
7,68c151d8-0d0d-11ea-bed8-ebcb87cb5715,PAYG,efa7004a-0d0c-11ea-b0b4-590f743c744f,White Toyota Yaris Petrol Hybrid (Automatic) -...,ND19 HWY,1211.0,Co-wheels,PAYG - Funded,Newbury - Boundary Road,NEW,591b7a98-2cd8-11ea-a341-71665a42c0e8,25692603,no,01/01/2020 21:30,01/01/2020 23:30,120,01/01/2020 21:29,01/01/2020 23:28,119.0,01/01/2020 21:30,01/01/2020 23:30,120,6.0,Co-wheels Standard Medium,2.0,0,0,11.00,1.08,11.0,12.08,0,-12.08,11.00,ended,no,01/01/2020 20:50,NaN,NaN
8,0fbf9512-b080-11e8-bea2-b13d9e3d63f4,PAYG,afa8ac2e-b082-11e8-8af1-3bdb9d1130f9,RETIRED - White Toyota Aygo X-Play (NL68 JVP),NL68 JVP,NaN,Co-wheels,PAYG,Swindon - Prospect Place,SWI,b6177928-2c55-11ea-a2e0-e9685b57c653,25692393,no,01/01/2020 21:15,01/01/2020 21:45,30,01/01/2020 21:22,01/01/2020 21:38,16.0,01/01/2020 21:15,01/01/2020 21:45,30,6.0,Co-wheels Standard Small,0.5,0,0,2.38,1.08,2.38,3.46,0,-3.46,3.46,ended,no,01/01/2020 5:15,NaN,NaN
13,c47d6fd2-8a1f-11e9-895d-dfa9f573edde,PAYG,f7bbebe0-7cd1-11e9-858f-1525255dc5c2,RETIRED - White Toyota Yaris Hybrid NJ68 JHV A...,NJ68 JHV,1106.0,Co-wheels,PAYG,Horsham - North Street,HOR,9ffdc790-27ba-11ea-a36d-c540c1997f03,5006643,no,01/01/2020 20:30,01/02/2020 20:30,1440,01/01/2020 22:02,01/02/2020 19:50,1308.0,01/01/2020 20:30,01/02/2020 20:30,1440,53.0,Co-wheels Standard Medium,0.0,1,0,38.50,9.54,38.5,48.04,0,-48.04,43.04,ended,yes,12/26/2019 8:35,NaN,NaN


Here, let's use 'location_description' column to fill values of 'location_office_use' (only if it has null values). This is because I want to use 'location_office_use' column to create 'location' column and also values of 'location_description' are not in the proper format. And even after handling missing values with this manual method there's missing values, then consider dropping those rows.

In [24]:
# Replace 'location_office_use' with 'BRI' where 'location_description' contains 'Lower Maudlin' or 'Bristol' and 'location_office_use' is null
bookings_filtered_df.loc[(bookings_filtered_df['location_description'].str.contains('Lower Maudlin|Bristol', case=False, na=False)) & (bookings_filtered_df['location_office_use'].isnull()), 'location_office_use'] = 'BRI'

# Replace 'location_office_use' with 'GLA' where 'location_description' contains 'glasgow' and 'location_office_use' is null
bookings_filtered_df.loc[(bookings_filtered_df['location_description'].str.contains('glasgow', case=False, na=False)) & (bookings_filtered_df['location_office_use'].isnull()), 'location_office_use'] = 'GLA'

# Replace 'location_office_use' with 'BIR' where 'location_description' contains 'Birmingham' and 'location_office_use' is null
bookings_filtered_df.loc[(bookings_filtered_df['location_description'].str.contains('Birmingham', case=False, na=False)) & (bookings_filtered_df['location_office_use'].isnull()), 'location_office_use'] = 'BIR'

# Replace 'location_office_use' with 'TUN' where 'location_description' contains 'Tunbridge Wells' and 'location_office_use' is null
bookings_filtered_df.loc[(bookings_filtered_df['location_description'].str.contains('Tunbridge Wells', case=False, na=False)) & (bookings_filtered_df['location_office_use'].isnull()), 'location_office_use'] = 'TUN'

# Replace 'location_office_use' with 'FRO' where 'location_description' contains 'Frome' and 'location_office_use' is null
bookings_filtered_df.loc[(bookings_filtered_df['location_description'].str.contains('Frome', case=False, na=False)) & (bookings_filtered_df['location_office_use'].isnull()), 'location_office_use'] = 'FRO'

# Replace 'location_office_use' with 'EXE' where 'location_description' contains 'Exeter' and 'location_office_use' is null
bookings_filtered_df.loc[(bookings_filtered_df['location_description'].str.contains('Exeter', case=False, na=False)) & (bookings_filtered_df['location_office_use'].isnull()), 'location_office_use'] = 'EXE'

# Replace 'location_office_use' with 'DUR' where 'location_description' contains 'Durham' and 'location_office_use' is null
bookings_filtered_df.loc[(bookings_filtered_df['location_description'].str.contains('Durham', case=False, na=False)) & (bookings_filtered_df['location_office_use'].isnull()), 'location_office_use'] = 'DUR'

# Replace 'location_office_use' with 'SAL' where 'location_description' contains 'Salford' and 'location_office_use' is null
bookings_filtered_df.loc[(bookings_filtered_df['location_description'].str.contains('Salford', case=False, na=False)) & (bookings_filtered_df['location_office_use'].isnull()), 'location_office_use'] = 'SAL'

# Replace 'location_office_use' with 'REA' where 'location_description' contains 'Reading' and 'location_office_use' is null
bookings_filtered_df.loc[(bookings_filtered_df['location_description'].str.contains('Reading', case=False, na=False)) & (bookings_filtered_df['location_office_use'].isnull()), 'location_office_use'] = 'REA'

In [25]:
# Drop rows if 'location_office_use' contains NaN value
bookings_filtered_df.dropna(subset=['location_office_use'], inplace=True)

Let's map 'location_office_use' with the actual name of the location (for example: ABI: Abingdon, NCL: Newcastle, etc.). However, there are three values in 'location_office_use': 'Not', 'RET, 'Ret which has different values in 'location_description', so different method was used to seperate 'location' from them.

In [26]:
# Dictionary for replacements
location_mapping = {
    'ABI': 'Abingdon', 'ABN': 'Aberdeen', 'BIC': 'Bicester', 'BIL': 'Billingshurst', 'BIR': 'Birmingham', 'BNB': 'Banbury', 'BOU': 'Bournemouth',
    'BRE': 'Brentwood', 'BRI': 'Bristol', 'CAN': 'Canterbury', 'CHI': 'Chichester', 'CHM': 'Chelmsford', 'CLY': 'Crawley', 'COA': 'Coatbridge',
    'DAL': 'Dalkeith', 'DDE': 'Dundee', 'DER': 'Derby', 'DUN': 'Dunbar', 'DUR': 'Durham', 'EAS': 'Eastbourne', 'EDI': 'Edinburgh', 'ELI': 'Elgin',
    'EST': 'Eastleigh', 'EXE': 'Exeter', 'EYN': 'Eynsham', 'FAL': 'Falkirk', 'FRO': 'Frome', 'GHD': 'Gateshead', 'GLA': 'Glasgow', 'HAD': 'Haddington',
    'HAI': 'Hainault', 'HAR': 'Harrogate', 'HAS': 'Hastings', 'HOR': 'Horsham', 'HOT': 'Henley-on-Thames', 'HRE': 'Houghton-Regis', 'HUN': 'Huntly',
    'HWY': 'High Wycombe', 'INR': 'Inverurie', 'IOW': 'Isle-of-Wight', 'IPS': 'Ipswich', 'KID': 'Kidlington', 'KNA': 'Knaresborough', 'KNT': 'Maidstone',
    'LAN': 'Lancaster', 'LEM': 'Leamington-Spa', 'LEW': 'Lewes', 'LON': 'Harrow', 'MUS': 'Musselburgh', 'NAN': 'Nantwich', 'NBE': 'North Berwick',
    'NCL': 'Newcastle', 'NEW': 'Newbury', 'NTH': 'North Shields', 'OHL': 'Oxenholme', 'ONF': 'On-fleet Bay', 'ORK': 'Orkney', 'OXF': 'Oxford',
    'PEN': 'Penrith', 'PER': 'Perth', 'PLY': 'Plymouth', 'PUT': 'Putney', 'REA': 'Reading', 'RIP': 'Ripon', 'SAF': 'Saffron Walden', 'SAL': 'Salford',
    'SBY': 'Salisbury', 'SHR': 'Shrewsbury', 'SOL': 'Solihull', 'SSH': 'South Shields', 'SUN': 'Sunderland', 'SWI': 'Swindon', 'TUN': 'Tunbridge Wells',
    'UPP': 'Upper Tooting', 'WAL': 'Walton-on-Thames', 'WAN': 'Wandsworth', 'WAR': 'Warwick', 'WIN': 'Winchester', 'WLG': 'Wallingford', 'WND': 'Windermere',
    'WNT': 'Wantage', 'WOK': 'Wokingham', 'WOR': 'Worthing', 'WRR': 'Warrington', 'WSM': 'Weston-super-Mare'
}

# Create 'location' column based on 'location_office_use'
bookings_filtered_df['location'] = bookings_filtered_df['location_office_use'].map(location_mapping)

# Identify the last column name
last_column = bookings_filtered_df.columns[-1]

# Create a new column order list
columns = bookings_filtered_df.columns.tolist()
columns.remove(last_column)
columns.insert(10, last_column)  # Insert the last column at the 7th position (index 6)

# Reorder the DataFrame columns
bookings_filtered_df = bookings_filtered_df[columns]

In [27]:
# Function to create 'location' column based on conditions
def create_location(row):
    if isinstance(row['location_office_use'], str) and any(keyword in row['location_office_use'] for keyword in ['Not', 'RET', 'Ret']):
        row['location'] = row['location_description'].split(' - ')[0].strip()
    return row

# Apply the function to create the 'location' column
bookings_filtered_df = bookings_filtered_df.apply(create_location, axis=1)

In [28]:
bookings_filtered_df.isnull().sum()

account_id                           0
Contract                             0
user_id                              0
vehicle_description                  0
vehicle_registration                 0
vehicle_communication_id         93802
vehicle_operator_name                0
vehicle_office_use                   0
location_description                 0
location_office_use                  0
location                             0
booking_id                           0
booking_reservation                  0
booking_maintenance                  0
booking_start                        0
booking_end                          0
booking_duration                     0
booking_actual_start                 0
booking_actual_end                   0
booking_actual_duration              0
booking_billed_start                 0
booking_billed_end                   0
booking_billed_duration              0
booking_mileage                     47
booking_tariff                       0
booking_rates_hours      

#### 1.3.5. Booking Mileage



---



Our dataset has insufficient features which are important for the analysis and modelling part of the problem statement. So, before moving ahead let's include those features in.

1. hourly_rate: Hourly rates for the car hire.
2. daily_rate: Daily rates for the car hire.
3. per_mile: Per mileage rates of the car.

In [ ]:
bookings_filtered_df.isna().sum()

account_id                           0
Contract                             0
user_id                              0
vehicle_description                  0
vehicle_registration                 0
vehicle_communication_id        139655
vehicle_operator_name                0
vehicle_office_use                   0
location_description             55285
location_office_use              56932
booking_id                           0
booking_reservation                  0
booking_maintenance                  0
booking_start                        0
booking_end                          0
booking_duration                     0
booking_actual_start                 0
booking_actual_end                   0
booking_actual_duration              0
booking_billed_start                 0
booking_billed_end                   0
booking_billed_duration              0
booking_mileage                     50
booking_tariff                       0
booking_rates_hours                  0
booking_rates_24hours    

In [ ]:
# Define hourly rates for different year-month combinations
hourly_rates = {
    '2019-01': 10.0,  # Example rate for January 2019
    '2019-02': 10.5,  # Example rate for February 2019
    '2020-01': 12.0,  # Example rate for January 2020
    '2020-02': 12.5,  # Example rate for February 2020
    # Add more rates for other year-month combinations if needed
}

bookings_filtered_df['hourly_rate'] =



---



In [ ]:
bookings_filtered_df[bookings_filtered_df['booking_mileage'].isnull()]

## 2. Remove Unuseful Columns

Drop columns that are irrelevant to solve problem statement.

In [ ]:
bookings_df.columns

Index(['account_id', 'Contract', 'user_id', 'vehicle_description',
       'vehicle_registration', 'vehicle_communication_id',
       'vehicle_operator_name', 'vehicle_office_use', 'location_description',
       'location_office_use', 'booking_id', 'booking_reservation',
       'booking_maintenance', 'booking_start', 'booking_end',
       'booking_duration', 'booking_actual_start', 'booking_actual_end',
       'booking_actual_duration', 'booking_billed_start', 'booking_billed_end',
       'booking_billed_duration', 'booking_mileage', 'booking_tariff',
       'booking_rates_hours', 'booking_rates_24hours',
       'booking_rates_overnight', 'booking_estimated_cost',
       'booking_actual_cost_distance', 'booking_actual_cost_time',
       'booking_actual_cost_total', 'booking_credits_used',
       'booking_transactions_value', 'booking_total_paid', 'booking_status',
       'booking_ended_early', 'booking_created_at', 'booking_cancelled_at',
       'booking_cancellation_reason'],
      dty

In [ ]:
# Remove unuseful columns from the dataframe

bookings_df.drop(['vehicle_registration', "vehicle_communication_id", "vehicle_operator_name", "vehicle_office_use", "booking_id", "booking_reservation",
               "booking_maintenance", "booking_start", "booking_end", "booking_duration", "booking_rates_overnight",
               "booking_estimated_cost", "booking_credits_used", "booking_transactions_value", "booking_total_paid", "booking_status", "booking_ended_early",
               "booking_created_at", "booking_cancelled_at", "booking_cancellation_reason"], axis=1, inplace=True)

In [ ]:
tariff_df

,Tariff,Size Category,Vehicle Type,PAYG or Contract,Petrol Or EV,Notes
0,All Saints View,Medium,Everyday,PAYG,EV,NaN
1,Bethesda Family EV,Large,Family,Franchise,EV,NaN
2,"Billing Exempt (24/7) (Default, Co-wheels)",Various,NaN,Contract,Various,NaN
3,Caernarfon Electric,Large,NaN,Franchise,EV,NaN
4,Canterbury Everyday,Medium,Everyday,PAYG,Petrol,NaN
...,...,...,...,...,...,...
84,Dundee City Council Rates (non-electric),Medium,NaN,Contract,Petrol,NaN
85,Hartlepool Standard Small,Small,NaN,Franchise,Petrol,NaN
86,McCarthy & Stone EV,Medium,NaN,Contract,EV,NaN
87,Sunderland City Council Rates,Various,NaN,Contract,Various,NaN


In [ ]:
tariff_df = tariff_df.rename(columns={"Tariff": "booking_tariff"})

In [ ]:
tariff_df.head()

In [ ]:
dff = pd.merge(bookings_filtered_df, tariff_df, how='outer', on='booking_tariff')

In [ ]:
dff.iloc[1]

In [ ]:
dff.head(5)

In [ ]:
# dff.to_excel(f'{DATA_PATH}/combined_file_outer.xlsx')

In [ ]:
dff[dff.isna()]

In [ ]:
dff.isna().sum()

In [ ]:
bookings_filtered.isna().sum()

### Data Analysis consists of:

1. Missing values
2. All Numerical Variables
3. Distribution of Numerical Variables
4. Categorical Variables
5. Cardinality of Categorical Variables
6. Outliers
7. Relationship between independant and Dependant Variable